In [ ]:
# [ ----- BASIC INSTALLS AND MODULE LOADING(S) ----- ]
!pip install faiss-cpu
!pip install sentence-transformers

import numpy as np
import torch
import sentence_transformers
import pandas
import json
from tqdm import tqdm

TRUE = True
FALSE = False
FRESH_START = FALSE
TEST_RUN = FALSE

Well this True False is very confusing, i do true or TRUE cuz cpp/winapi so i defined those; should ctrl f -> replace all TRUE with True and same for false but if i work on it in future, most probably will face this again so im keeping it.

In [ ]:
# [ ----- GET BNS DATASET TO BUILD RAG AROUND ----- ]
# this cell if for loading kaggle and getting BNS dataset
if FRESH_START:
    !mkdir -p ~/.kaggle
    !mv kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json

    !kaggle datasets download -d nandr39/bharatiya-nyaya-sanhita-dataset-bns
    !unzip bharatiya-nyaya-sanhita-dataset-bns.zip -d bns_data


**Since the dataset is so big, I saved one copy in drive for now for, fast upload**

SOURCE : https://www.kaggle.com/datasets/adarshsingh0903/legal-dataset-sc-judgments-india-19502024

In [ ]:
# [ ----- GET SUPEREME COURT TO BUILD RAG AROUND ----- ]
# this cell is for {loading kaggle} and supereme courts' dataset

if FRESH_START:
    !cp /content/drive/MyDrive/colab_stuff/court_hearings_rag/archive.zip /content/
    !unzip archive.zip -d /content/sc_judgments


The Sections below are all responsible for visiting the supereme courts' dataset(unzipped and script.py deleted) file by file and extracting relevant data, reading pdf(s) and saving them as a file

Last time this was run(and first time) it took ~16 hours, so better use the saved, parsed data from drive

I also happened to write a multiprocessing version for this parsing and saving pdf(s) only to realize colab is gonna give me only 1 process to work with anyways. Also i found out multiprocessing is slower in windows than linux(the type i needed which made a process for 1 function, ran it, returned value, died, over and over).

I also considered using local machine for this, R5 7535HS, which is afterall, 6C 12T capable processor onnly to realize me using multi core(s) is still somehow slower than what single process of colab and achieve, in terms of speed, hence not pasting that code here. If intereseted, can see it in prev upload, when whole project was single notebook.

In [ ]:
# [ ------ For Saving Supereme courts' dataset cuz its too big to load everytime ----- ]
# we save each case data like this for later convinience, called in single core one
path_to_json = '/content/apex.jsonl' # advice gotten use json line, advice accepted
all_cases = []

def save_it(pages, file, file2, id):
    temp = {
        'id' : id,
        'source' : "Supreme Court",
        'title' : file2,
        'text' : pages,
        'meta' : {
            'year' : file
            }
    }
    with open(path_to_json, "a", encoding='utf-8') as f:
        f.write(json.dumps(temp, ensure_ascii=False)) # dumps returns string of what dump was supposed to save
                                                    # we need to write line by line hence the way of f.write(json.dumps())
        f.write("\n") # this is how we make jsonl mate

all_cases.clear()

In [ ]:
# [ ----- Responsible for removing headers and footers' of pdf of supereme court judgements ----- ]
def _apex_remove_header_footer(text):
    HEADER = None
    FOOTER = f'Indian Kanoon - '
    HEADER, _ = current_case['text'].split('\n', 1) # in the text region the first line before '\n' gives us header, so only 1 split needed
    cleaned_current_case = []

    for line in current_case['text'].split('\n'):
        if line.startswith(HEADER) or line.startswith(FOOTER):
            continue
        else:
            cleaned_current_case.append(line)
            # a weird observation, even after loads() my footer of this and header of next page don't have \n in between
            # so like just detecting footers is enough to remove headers as well
    return '\n'.join(cleaned_current_case)  # we split on '\n' so we join on '\n\

In [ ]:
# [ ----- For cleaning subtle things bcs pdf has page so words get cut ----- ]
# OBSERVATION 1 - if a sentance ends with \n but no fullstop before it, it was cut midway, remove \n
# OBSERVATION 2 - if a word has -\n it was cut midway, remove -\n
# OBSERVATION 3 - multiple -- serve some purpose, wait now, remove later
def _remove_enter_mid_words(text):
    formatted_sentance = []
    sentances = text.split('. ')
    for sentance in sentances:
        sentance = sentance.replace('-\n', '')
        sentance = sentance.replace('\n', ' ')
        formatted_sentance.append(sentance)
    return '.\n'.join(formatted_sentance)

In [ ]:
'''SINGLE CORE SINGLE CORE SINGLE CORE SINGLE CORE SINGLE CORE SINGLE CORE SINGLE CORE'''
import os
apex_raw = []

if FRESH_START:
    import pdfplumber
    ROOT = "/content/sc_judgments/supreme_court_judgments"
    start_id = sum(1 for _ in open(path_to_json, "r", encoding="utf-8"))
    last_line = None

    print(f'starting fro i :: {start_id}')
    i = 1
    for file in sorted(os.listdir(ROOT)): # order of traversal is weird without sorted
        # print(file)
        print(f"\ncurrently in year :: {file}\n currently in case :: ", end=' ') # so i don't die waiting
        for file2 in sorted(os.listdir(f"{ROOT}/{file}")):
            if i < start_id:
                i += 1
                continue

        print(f"{i}", sep=' ', end=' ')
        try:
            with pdfplumber.open(f"{ROOT}/{file}/{file2}") as pdf:
                pages = []
                for page in pdf.pages:
                    try:
                        text = page.extract_text()
                        if text:
                            pages.append(text)
                            # print(pages)
                    except Exception:
                        print(f'something wrong with page in pdf :: {i} ')
                        with open('/content/errors.jsonl', 'a') as f:
                            f.write(json.dumps(f' PAGE PROBLEM :: PDF :: {i} Year :: {file} Title :: {file2}'))
                            f.write('\n')
                            f.close()
                    # only after pages has been appended with all data
                save_it(pages, file, file2, i)
                i += 1
        except Exception as e: # are these always meant to be used or always realized the need for
            print(f'something wrong with pdf :: {i}')
            with open('/content/errors.jsonl', 'a') as f:
                f.write(json.dumps(f' PDF PROBLEM :: PDF :: {i} Year :: {file} Title :: {file2}'))
                f.write('\n')
                f.close()
        !cp {path_to_json} /content/drive/MyDrive/colab_stuff/court_hearings_rag/apex.jsonl
        print(f'\n:: copied to drive ::\n')
else:
    !cp /content/drive/MyDrive/colab_stuff/court_hearings_rag/apex.jsonl {path_to_json}
    with open(path_to_json, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            current_case = json.loads(line)
            current_case['text'] = _remove_enter_mid_words(_apex_remove_header_footer(current_case['text']))
            apex_raw.append(current_case)


Source for BNS :: https://www.kaggle.com/datasets/nandr39/bharatiya-nyaya-sanhita-dataset-bns

Initially all dataset had custom representation because each dataset unique, but over time i was getting irritated writing functions for each of their handling so i made a cell to unify everyones' representation to be the same after apex dataset was done with. Though i want to remember these things so i keep it/preserve as much of originality/suffering/story as possible

In [ ]:
# [LOAD BNS DATASET]
SAVE_PATH1 = "/content/drive/MyDrive/colab_stuff/court_hearings_rag/bns.json"

if FRESH_START:
    bns_raw = pandas.read_csv("/content/bns_data/bns_sections.csv")
    print(bns_raw.head())
    bns = []
    for i, entries in bns_raw.iterrows():
        temp = {
        'id' : i+1,
        'chapter': entries['Chapter'],
        'section': entries['Section'],
        'title': entries['Section _name'],
        'text': entries['Description']
        }
        bns.append(temp)
    print(bns[0])

    with open(SAVE_PATH1, 'w') as f:
        json.dump(bns, f)
else:
    with open(SAVE_PATH1, 'r') as f:
        bns = json.load(f)

In [ ]:
def _clean_bns_data(data):
    sentances = data['text'].split('\r\n')
    data['text'] = '\n'.join(sentances)
    return data

for data in bns:
    data = _clean_bns_data(data)


Get high court and supereme courts' dataset of which i won't use supereme court cuz i have better dataset for that. I will filter only high court cases from this

In [ ]:
if FRESH_START:
    TOKEN = ""
    dataset_supereme_high_court = datasets.load_dataset("opennyaiorg/InJudgements_dataset", token=TOKEN)


This dataset seems like scraped from web containing several html fields so gotta remove them by regex, and it contains decorations in forms of ===== so get rid of that as well


In [ ]:
# [LOAD SUP-HI COURT DATASET]
SAVE_PATH2 = "/content/drive/MyDrive/colab_stuff/court_hearings_rag/high_courts.json"

# [THE TEXT DATA IS TOO NOISY WITH HTML TAGS]
def clean_html(text):
    import re
    text = re.sub(r"<span[^>]*>.*?</span>", " ", text, flags=re.DOTALL) # REMOVE 'EM HTML
    text = re.sub(r"<[^>]+>", " ", text) # REMOVE 'EM HTML
    text = re.sub(r"[=\-_.]{4,}", " ", text)

    return text

# print(dataset_supereme_high_court)
# print(dataset_supereme_high_court['train'][0])
if FRESH_START:
    high_courts_raw = dataset_supereme_high_court['train']
    # high_courts_raw[0]
    # print(high_courts_raw[0].keys())
    high_courts = []
    for i, cases in enumerate(high_courts_raw):
        temp = {
            'id' : i+1,
            'court_details' : f"{cases['Court_Type']} | {cases['Case_Type']} | {cases['Court_Name']}",
            'cited_by' : cases['Cited_by'],
            'title': cases['Titles'],
            'text' : clean_html(cases["Text"])
        }
        high_courts.append(temp)

    print(high_courts[0])
    with open(SAVE_PATH2, "w") as f:
        json.dump(high_courts, f)
else:
    with open(SAVE_PATH2, "r") as f:
        high_courts = json.load(f)

In [ ]:
# [ ----- COnsider only high courts' decisions from this dataset ----- ]
# well let us drop all supereme court cases from here and use it has high court dataset

high_courts = [high_court for high_court in high_courts if high_court['court_details'].startswith('High')]


In [ ]:
def _make_high_court_paragraphs(data):
    cleaned_lines = []
    # print(f":: toatal paragraphs made {len(data['text'].split('\n\n'))} :: ")
    length = 0
    for line in data['text'].split('\n\n'):
        line = " ".join(line.split())
        cleaned_lines.append(line)
        length = max(length, len(line))
    # print(f"longest paragraphs' length is {length}")
    data['text'] = '\n'.join(cleaned_lines)
    return data

print(high_courts[0])

for cases in high_courts:
    cases = _make_high_court_paragraphs(cases)

# print(high_courts[0])
# print(high_courts[0:2])

Yes, by this time i had experienced too much pain loading datasets so many times having to deal with two different dictionary for 2 dataset that i decided to finally standardize dictionary keys accross datasets' representations

In [ ]:
"""
[UNIFYING ALL DATA REPRESENTATIONS]
well i see no point in doing this, but apprearently it helps later on figuring what went wrong in
RAG design and extending it in future as well.
"""

new_bns = []
for entries in bns:
    temp = {
        'id' : f"BNS_{entries['id']}",
        'source' : "BNS",
        'title' : entries['title'],
        'text' : entries['text'],
        'meta' : {
            'chapter' : entries['chapter'],
            'section' : entries['section']
            }
    }
    new_bns.append(temp)

bns = new_bns
del new_bns

new_court = []
for entries in high_courts:
    temp = {
        'id' : f"COURTS_{entries['id']}",
        'source' : "Courts",
        'title' : entries['title'],
        'text' : entries['text'],
        'meta' : {
            'cited_by' : entries['cited_by'],
            'court_details' : entries['court_details']
            }
    }
    new_court.append(temp)
court = new_court # yeh i changed name.
del new_court
del high_courts
print(court[0])
print("\n\n")
print(bns[0])

Just checking if things are progressingn as expected-

all the keys of dictionary are correct
all dataset contains instances of \n for later chunkning step
no useless space or fancy symbol left in text field

some problems does exist like .pdf in title or short title in bns but by this point in time i don't have real use of all these metadata so i will ignore them

In [ ]:
# VISUAL VERIFICATION
print(court[0])
print("\n")
print(bns[0])
print("\n")
print(apex_raw[0])
print("\n\n")
print(court[0]['text'][:7000])
print("\n")
print(bns[0]['text'][:700])
print("\n")
print(apex_raw[0]['text'][:700])

For testing purposes, i will take a small sample of data to run cells below fast so i don't have to wait hours. This is just to verify the pipeline after this, as they take hours for embedding and chunking

In [ ]:
if TEST_RUN:
    apex_raw = apex_raw[:50]
    court = court[:50]
    bns = bns[:50]

CHUNKING - the pain and grail of RAG. This is where i expect most improvements to come from. Chunk length is 300 only cuz long chunks was getting lots of different context making it very noisy and it was too long to feed into model(s) later. Rather have small relevant high precision chunks and retrieve appropriate data based on metadata sstored with it, i mwan why even was that stored anyways otherwise.

In [ ]:
def split_long_paragraph(paragraph, max_words):
    words = paragraph.split()
    parts = []
    for i in range(0, len(words), max_words):
        parts.append(" ".join(words[i:i + max_words]))
    return parts

def _label_chunks(chunks, data):
    chunks_final = []
    for i, chunk in enumerate(chunks):
        temp = {
            'chunk_id': f"{data['source']}_{data['id']}_{i}",
            'chunk_text': "\n".join(chunk),
            'parent_id': data['id'],
            'source': data['source']
        }
        chunks_final.append(temp)
    return chunks_final

def make_chunks(data, tokenizer, para_overlap=1, max_tokens=800, token_overlap=150):

    paragraphs = [p.strip() for p in data["text"].split("\n") if p.strip()]
    chunks = []
    chunk = []
    running_len = 0
    expanded_paragraphs = []

    # word-based limit derived from token budget
    max_words = int(0.7 * max_tokens)

    # long para(s) we have a lot of these unfortunately
    for p in paragraphs:
        p_len = len(tokenizer.encode(p, add_special_tokens=False))

        if p_len > max_tokens:
            parts = split_long_paragraph(p, max_words)
            for part in parts:
                part_len = len(tokenizer.encode(part, add_special_tokens=False))
                expanded_paragraphs.append((part, part_len))
        else:
            expanded_paragraphs.append((p, p_len))
    # -----------------------------------------------

    paragraphs = [p for p, _ in expanded_paragraphs]
    para_tokens = [l for _, l in expanded_paragraphs]

    for i, paragraph in enumerate(paragraphs):
        p_len = para_tokens[i]

        # if adding paragraph would overflow
        if running_len + p_len > max_tokens:
            if chunk:
                chunks.append(chunk.copy())

            # start new chunk with overlap
            chunk = []
            running_len = 0

            if para_overlap > 0:
                for j in range(max(0, i - para_overlap), i):
                    overlap_len = para_tokens[j]
                    if running_len + overlap_len > token_overlap:
                        break
                    chunk.append(paragraphs[j])
                    running_len += overlap_len

        chunk.append(paragraph)
        running_len += p_len

    if chunk:
        chunks.append(chunk.copy())

    return _label_chunks(chunks, data)


this tqdm is godsent, i now know not knowing how long to wait is more painful than having to wait 16 hours for all pdf to be parsed. And i switched from jsons to pickle for my preffered method of storing data cuz iz python aftrall.

In [ ]:
from tqdm import tqdm
import pickle

model = sentence_transformers.SentenceTransformer('all-MiniLM-L6-v2')
tokenizer = model.tokenizer
chunks_bns = []
chunks_high = []
chunks_apex = []
if FRESH_START:
    print(f':: currently doing BNS :: ')
    for cases in tqdm(bns):
        labeled_chunk = make_chunks(cases, tokenizer, 0, 300, 0)
        chunks_bns += labeled_chunk
    with open('/content/bns_chunked.pkl', 'wb') as f:
        pickle.dump(chunks_bns, f)
        !cp /content/bns_chunked.pkl /content/drive/MyDrive/colab_stuff/court_hearings_rag/bns_chunked.pkl
    print(f" :: currently doing high_courts :: ")

    for cases in tqdm(court):
        labeled_chunk = make_chunks(cases, tokenizer, 1, 300, 65)
        chunks_high += labeled_chunk
    with open('/content/high_chunked.pkl', 'wb') as f:
        pickle.dump(chunks_high, f)
        !cp /content/high_chunked.pkl /content/drive/MyDrive/colab_stuff/court_hearings_rag/high_chunked.pkl
    print(f' :: currently doing apex :: ')

    for cases in tqdm(apex_raw):
        labeled_chunk = make_chunks(cases, tokenizer, 1, 300, 65)
        chunks_apex += labeled_chunk
    with open('/content/apex_chunked.pkl', 'wb') as f:
        pickle.dump(chunks_apex, f)
        !cp /content/apex_chunked.pkl /content/drive/MyDrive/colab_stuff/court_hearings_rag/apex_chunked.pkl
else:
    with open('/content/drive/MyDrive/colab_stuff/court_hearings_rag/apex_chunked.pkl', 'rb') as f:
        chunks_apex = pickle.load(f)

    with open('/content/drive/MyDrive/colab_stuff/court_hearings_rag/high_chunked.pkl', 'rb') as f:
        chunks_high = pickle.load(f)

    with open('/content/drive/MyDrive/colab_stuff/court_hearings_rag/bns_chunked.pkl', 'rb') as f:
        chunks_bns = pickle.load(f)


In [ ]:
print(f'{len(chunks_apex)}')
print(f'{len(chunks_high)}')
print(f'{len(chunks_bns)}')

EMBEDDING :: well then we need to embed the chunks thus made

In [ ]:
'''
make embedding and select model
'''
model_name = 'all-MiniLM-L6-v2'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = sentence_transformers.SentenceTransformer(model_name, device=device)
# remember that faiss is cpu first, need to move everything to gpu manually

def embed(model, embedding_name, chunks):
    # model = sentence_transformers.SentenceTransformer(model_name, device=device)
    path = f'/content/embedding_{embedding_name}.npy'
    print(f'currently embedding za {embedding_name}')

    if FRESH_START:
        # model.to(device) not adviced, do at creation, else things break
        chunk_texts = [chunk['chunk_text'] for chunk in chunks]
        embeddings = model.encode(
            chunk_texts,
            batch_size = 64,
            convert_to_numpy=True, # remember faiss numpy
            show_progress_bar=True,
            normalize_embeddings=True,
            device=device
            )
        np.save(path, embeddings)
        !cp {path} /content/drive/MyDrive/colab_stuff/court_hearings_rag/{embedding_name}.npy
    else:
        embeddings = np.load(path)
    return embeddings

In [ ]:
if FRESH_START:
    embedding_bns = embed(model, 'bns_embed', chunks_bns)
    embedding_apex = embed(model, 'apex_embed', chunks_apex)
    embedding_high = embed(model, 'high_embed', chunks_high)
else:
    embedding_bns = np.load('/content/drive/MyDrive/colab_stuff/court_hearings_rag/bns_embed.npy')
    embedding_apex = np.load('/content/drive/MyDrive/colab_stuff/court_hearings_rag/apex_embed.npy')
    embedding_high = np.load('/content/drive/MyDrive/colab_stuff/court_hearings_rag/high_embed.npy')


And finally, we have our FAISS, this thing has soooo much potential, i gotta explore this in much greater depth in future.

In [ ]:
import faiss

dim = embedding_bns.shape[1]

index_bns = faiss.IndexFlatIP(dim)
index_apex = faiss.IndexFlatIP(dim)
index_high = faiss.IndexFlatIP(dim)

index_bns.add(embedding_bns)
index_apex.add(embedding_apex)
index_high.add(embedding_high)

The loading from drive part worked before, but now that i have more embeddings because i changed embedding size, it doesn't work cuz appearently

Well the probelem is not exactly size but faiss, i am trying to save faiss indices, and faiss is c/cpp backend not python. It contains 'non python' thing which pickle is not so much comfortable be, for instance, pointers allocated outside python heap or native memory buffers, so things break.

So the only option is to rebuild index every iteration using th eembeddings.npy made.

well faiss expects query to be list and accordingly returns similarity score and index which are both 2d list, 1 dimention for each query and second for each vector returned for query. Thus we loop over both for retrieval

In [ ]:
def search_info(query, model, metadata, top_k=5):
    # first use the same sentance embedding model to make query database compatible
    query_vector = model.encode(query, normalize_embeddings=True, convert_to_numpy=True)

    sim_1, idx_1 = index_bns.search(query_vector, top_k)
    sim_2, idx_2 = index_apex.search(query_vector, top_k)
    sim_3, idx_3 = index_high.search(query_vector, top_k )

    chunks_bns, chunks_apex, chunks_high = metadata
    print(len(query))
    for q in range(len(query)):
        for i in range(top_k):
            print(f'{i+1}th most similar to query in bns with similarity :: {sim_1[q][i]} is :: \n{chunks_bns[idx_1[q][i]]}')
            print(f'{i+1}th most similar to query in apex with similarity :: {sim_2[q][i]} is :: \n{chunks_apex[idx_2[q][i]]}')
            print(f'{i+1}th most similar to query in high with similarity :: {sim_3[q][i]} is :: \n{chunks_high[idx_3[q][i]]}')


query = [
    "Can a criminal conviction be sustained solely on the testimony of a single eyewitness?",
    "Is delay in filing an FIR a sufficient ground to discard the prosecution case?"
    ]

metadata = (chunks_bns, chunks_apex, chunks_high)
search_info(query, model, metadata, 2)

In [ ]:
# Confirming about whether faiss and chunked_* are same data
print(f'length of chunks_bns is {len(chunks_bns)} :: embedded = {len(embedding_bns)}')
print(f'length of chunks_bns is {len(chunks_apex)} :: embedded = {len(embedding_apex)}')
print(f'length of chunks_bns is {len(chunks_high)} :: embedded = {len(embedding_high)}')

In [ ]:
import pickle
with open('/content/chunk_text.pkl', 'wb') as f:
    pickle.dump(metadata, f) # no need for encoding(s) when working in binary
!cp /content/chunk_text.pkl /content/drive/MyDrive/colab_stuff/court_hearings_rag/chunk_text.pkl

The faiss part is repeated in next one as well, since i need to build faiss again cuz c++. But i am taking metadata, which is not metadata but that is thae name coming to my mind everytime i think of it so i name it metadata. I save it to file and use it in next notebook, this won't break, as metadata is made of all python elements so pickle will pickle it.